In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/revealed_test.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv
/kaggle/input/usa-counties-coordinates/cfips_location.csv


In [2]:
godaddy_census = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv')

In [3]:
print(godaddy_census.columns)

Index(['pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020',
       'pct_bb_2021', 'cfips', 'pct_college_2017', 'pct_college_2018',
       'pct_college_2019', 'pct_college_2020', 'pct_college_2021',
       'pct_foreign_born_2017', 'pct_foreign_born_2018',
       'pct_foreign_born_2019', 'pct_foreign_born_2020',
       'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018',
       'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021',
       'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019',
       'median_hh_inc_2020', 'median_hh_inc_2021'],
      dtype='object')


In [4]:
godaddy_census.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


In [5]:
godaddy_revealed = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/revealed_test.csv')
godaddy_train = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')

In [6]:
print(godaddy_train.columns)

Index(['row_id', 'cfips', 'county', 'state', 'first_day_of_month',
       'microbusiness_density', 'active'],
      dtype='object')


In [7]:
print(godaddy_revealed.columns)

Index(['row_id', 'cfips', 'county', 'state', 'first_day_of_month',
       'microbusiness_density', 'active'],
      dtype='object')


In [8]:
godaddy_train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [9]:
godaddy_train.shape

(122265, 7)

In [10]:
godaddy_train = pd.concat([godaddy_train,godaddy_revealed]).sort_values(by = ['cfips','first_day_of_month']).reset_index()

In [11]:
godaddy_train.shape

(128535, 8)

In [12]:
godaddy_test = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv')

In [13]:
godaddy_test.shape

(25080, 3)

In [14]:
print(godaddy_test.columns)

Index(['row_id', 'cfips', 'first_day_of_month'], dtype='object')


In [15]:
godaddy_test.head()

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01


In [16]:
drop_index = (godaddy_test.first_day_of_month == '2022-11-01') | (godaddy_test.first_day_of_month == '2022-12-01')

In [17]:
godaddy_test = godaddy_test.loc[~drop_index,:]

In [18]:
godaddy_test.shape

(18810, 3)

In [19]:
godaddy_test.head()

,row_id,cfips,first_day_of_month
6270,1001_2023-01-01,1001,2023-01-01
6271,1003_2023-01-01,1003,2023-01-01
6272,1005_2023-01-01,1005,2023-01-01
6273,1007_2023-01-01,1007,2023-01-01
6274,1009_2023-01-01,1009,2023-01-01


In [20]:
godaddy_coordinates  = pd.read_csv('/kaggle/input/usa-counties-coordinates/cfips_location.csv')

In [21]:
godaddy_coordinates.head()

,cfips,name,lng,lat
0,1059,Franklin,-87.843283,34.442381
1,13111,Fannin,-84.319296,34.864126
2,19109,Kossuth,-94.206898,43.204140
3,40115,Ottawa,-94.810589,36.835878
4,42115,Susquehanna,-75.800905,41.821277


In [22]:
godaddy_train['istest'] = 0
godaddy_test['istest'] = 1

In [23]:
raw_data = pd.concat((godaddy_train,godaddy_test)).sort_values(['cfips','row_id']).reset_index(drop=True)
raw_data = raw_data.merge(godaddy_coordinates.drop("name",axis = 1),on="cfips")

In [24]:
raw_data.head()

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,lat
0,0.0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249.0,0,-86.6429,32.535142
1,1.0,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,-86.6429,32.535142
2,2.0,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,-86.6429,32.535142
3,3.0,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,-86.6429,32.535142
4,4.0,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,-86.6429,32.535142
